In [1]:
import numpy as np
from numpy.testing import assert_allclose
from keras.models import Sequential, load_model
from keras.layers import LSTM, Dropout, Dense
from keras.callbacks import ModelCheckpoint
!pip install plotly --upgrade  # make sure your plotly is up to date

import plotly.express as px
import numpy as np
import pandas as pd
from scipy.io import loadmat
import matplotlib.pyplot as plt

Requirement already up-to-date: plotly in /usr/local/lib/python3.7/dist-packages (4.14.3)


In [2]:
from google.colab import drive
drive.mount('/content/drive')

spectra = loadmat('/content/drive/MyDrive/Spectra_3s_25c.mat')
#more_spectra = loadmat('/content/drive/MyDrive/5_more_spectra.mat')
spectra.keys()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


dict_keys(['__header__', '__version__', '__globals__', 'AmpSAmp', 'Dm_total_save', 'GammaAmp', 'GammaSAmp', 'GammaS_total_save', 'Gamma_total_save', 'M', 'NmaxS', 'dG', 'dGS', 'frequencies', 'n_int', 'noiseAmpScaled', 'omegaS_total_save', 'omega_total_save'])

In [3]:
new_model = load_model('/content/drive/MyDrive/model_complete.h5')

In [4]:
all_channels = pd.DataFrame(np.transpose(np.transpose(spectra['Dm_total_save'])[0]))
modes = [x[0]*np.shape(spectra['Dm_total_save'])[0] for x in spectra['omega_total_save']]

In [5]:
labeled_modes=[]
for i in range(3):
  print("ITERATION: "+str(i))
  amp=[]
  length = np.shape(spectra['Dm_total_save'])[0]
  modes = [x[i]*length for x in spectra['omega_total_save']]
  seeker = 0
  modes = np.sort(modes)
  for j in range(length):
    all_amp = [0]
    while seeker < len(modes) and modes[seeker] <= j+100:
        # print(np.exp(np.square((j-modes[seeker])/100)))
      all_amp.append(1/(np.exp(np.square((j-modes[seeker])/10))))
      seeker += 1
    while seeker < len(modes) and seeker >= 0 and modes[seeker] >= j-100:
      # print(modes[seeker])
      seeker -= 1
    seeker = max(seeker,0)
    amp.append(max(all_amp))
    if j %100000 ==0: print(j)
  for i in range(5):
    labeled_modes.extend(amp)
  print(len(labeled_modes))

ITERATION: 0
0


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: RuntimeWarning: overflow encountered in exp
  del sys.path[0]


100000
200000
300000
400000
500000
600000
700000
800000
900000
5000000
ITERATION: 1
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
10000000
ITERATION: 2
0
100000
200000
300000
400000
500000
600000
700000
800000
900000
15000000


In [6]:
# training set generation
x = []
for i in range(3):
  all_channels = pd.DataFrame(np.transpose(np.transpose(spectra['Dm_total_save'])[i]))
  x.extend(np.array(all_channels[[0,1,2,3,4]]))
  x.extend(np.array(all_channels[[5,6,7,8,9]]))
  x.extend(np.array(all_channels[[10,11,12,13,14]]))
  x.extend(np.array(all_channels[[15,16,17,18,19]]))
  x.extend(np.array(all_channels[[20,21,22,23,24]]))

In [7]:
!pip freeze | grep plotly

plotly==4.14.3


In [8]:
pd.options.plotting.backend = "plotly" 

def show_spectra(view_range,spect,modes, spectra_sample=True):
  # take subset of plot
  t_modes = np.sort(modes)
  temp_modes = pd.Series(t_modes)[(pd.Series(t_modes) < view_range[1])]
  temp_modes = pd.Series(temp_modes)[(pd.Series(temp_modes) > view_range[0])]
  fig = []
  if spectra_sample:
    fig = pd.Series(spect).iloc[range(view_range[0],view_range[1])].plot(kind='line')
  else:
    fig = pd.Series(spect).plot(template='plotly_dark', kind='line')
    temp_modes -= view_range[0]
  for mode in temp_modes:
    fig.add_vline(x=mode, line_width=1, line_color="green") #requires plotly 4.12 and above
  fig.show()

In [9]:
modes = [x[0]*length for x in spectra['omega_total_save']]
show_spectra((int(500000), int(501000)), labeled_modes, modes)

In [10]:
def stop():
  spectra =  loadmat('Spectra.mat')#['Dm_total_save']

  labeled_modes=[]
  for i in range(1):
    print("ITERATION: "+str(i))
    amp=[]
    length = np.shape(spectra['Dm_total_save'])[0]
    modes = [x[i]*length for x in spectra['omega_total_save']]
    seeker = 0
    modes = np.sort(modes)
    for j in range(length):
      all_amp = [0]
      while seeker < len(modes) and modes[seeker] <= j+100:
          # print(np.exp(np.square((j-modes[seeker])/100)))
        all_amp.append(1/(np.exp(np.square((j-modes[seeker])/10))))
        seeker += 1
      while seeker < len(modes) and seeker >= 0 and modes[seeker] >= j-100:
        # print(modes[seeker])
        seeker -= 1
      seeker = max(seeker,0)
      amp.append(max(all_amp))
      if j %100000 ==0: print(j)
    labeled_modes.extend(amp)
    print(len(labeled_modes))

#y = labeled_modes
#x = spectra['Dm_total_save']
#window_len = 1000
#reshaped_y = np.reshape(y, (int(len(y)/window_len), window_len))
#reshaped_x = np.reshape(np.array(x), (int(len(x)/window_len), window_len, 5))

In [11]:
y = labeled_modes
window_len = 1000
reshaped_y = np.reshape(y, (int(len(y)/window_len), window_len))
reshaped_x = np.reshape(np.array(x), (int(len(x)/window_len), window_len, 5))

In [12]:
first_spectra = reshaped_x[0:1000]

In [13]:
y_pred = new_model.predict(first_spectra)

In [14]:
np.reshape(y_pred, (1, 1000000))

#modes = [x[0]*length for x in spectra['omega_total_save']]
show_spectra((int(500000), int(501000)), np.reshape(y_pred, (1, 1000000))[0], modes)

In [15]:
show_spectra((int(500000), int(501000)), y, modes)

In [16]:
import seaborn as sns
from scipy.signal import savgol_filter
from scipy.signal import find_peaks

reshaped_y_pred = np.reshape(y_pred, (1, 1000000))[0]
#pd.Series(np.reshape(reshaped_y, (1, 1000000))[0][0:10000]).plot()
#pd.Series(savgol_filter(y_pred, 51, 3)[0:10000]).plot()
predicted_modes = find_peaks(reshaped_y_pred, height=0.65)[0]
#print(len(modes))
#sns.lineplot(data = pd.Series(np.reshape(reshaped_y, (1, 1000000))[0][0:10000]))

In [17]:
print(len(predicted_modes))

10085


In [18]:
modes = [x[0]*length for x in spectra['omega_total_save']]

In [19]:
peaks_detected_count = []
peaks_detected = []
for s in range(10):
  window_peaks = [i for i in predicted_modes if (i >= s * 100000) and (i < (s+1) * 100000)]#find_peaks(smooth_density[s*100000 : (s+1)*100000], prominence=density_prominence)
  peaks_detected_count.append(len(window_peaks))
  window_peaks = [i for i in modes if (i >= s * 100000) and (i < (s+1) * 100000)]
  peaks_detected.append(len(window_peaks))
  #peaks_detected.extend(window_peaks+s*100000)
print("number of peaks detected:", str(np.sum(peaks_detected_count)))

number of peaks detected: 10085


In [20]:
peaks_detected_count

[122, 323, 434, 768, 938, 1146, 1274, 1494, 1748, 1838]

In [21]:
pd.Series(predicted_modes).plot.hist()
# pd.Series(modes).plot.hist()

In [22]:
pd.Series(modes).plot.hist()

In [23]:
# a_l = [0]*len(a)
# b_l = [0]*len(b)
# threshold = .1
# for i in range(len(dists)):
#   while dist < threshold:
#     if a[i] == 0:
#       row_min = np.argmin(dist)//dist.shape[1]
#       col_min = np.argmin(dist)%dist.shape[1]
#       a[i] = col_min
#       b[i] = row_min
      
    

In [24]:
modes = np.array(modes)
modes.sort()

In [25]:
modes

array([  1190.20695012,   3183.28463774,   3444.60805029, ...,
       999582.11096774, 999721.85571522, 999893.73555854])

tp = num_pairs

fp = in pred modes but not in modes = len(predicted_modes) - num_pairs

tn = does not exist

fn = in modes but not in pred modes = len(modes) - num_pairs

In [27]:
# Insert sorted np.arrays of modes and predicted modes
def get_correct_preds(modes, predicted_modes, threshold = 50):
  mode_dists = np.abs(predicted_modes[:, np.newaxis] - modes)
  dist_df = pd.DataFrame(mode_dists.flatten(), columns = ['distance'])
  dist_df['predicted_index'] = np.indices((mode_dists.shape[0], mode_dists.shape[1]))[0].flatten()
  dist_df['actual_index'] = np.indices((mode_dists.shape[0], mode_dists.shape[1]))[1].flatten()

  filter_df = dist_df[dist_df['distance']<50]
  sort_df = filter_df.sort_values(filter_df.columns[0])
  dist_arr = sort_df.to_numpy()

  p_actual= []
  p_pred = []
  num_pairs = 0
  for item in dist_arr:
    if item[1] not in p_pred and item[2] not in p_actual:
      p_pred.append(item[1])
      p_actual.append(item[2])
      num_pairs += 1

  return num_pairs, p_actual, p_pred

In [35]:
def get_precision_recall_f1(num_pairs, modes, predicted_modes):
  fp = len(predicted_modes) - num_pairs
  fn = len(modes) - num_pairs
  precision = num_pairs / (num_pairs + fp)
  recall = num_pairs / (num_pairs + fn)
  f1 = 2 * precision * recall / (precision + recall)
  return precision, recall, f1

In [32]:
final_neighbors = get_correct_preds(modes, predicted_modes, 50)
tp = final_neighbors[0]
tp

6589

In [36]:
get_precision_recall_f1(tp, modes, predicted_modes)

(0.6533465542885474, 0.6589, 0.6561115260144386)